In [2]:
import sqlite3
import pandas as pd

MMEX_PATH: str = "/home/paolo/Nextcloud/MoneyManager/finances.mmb"

In [3]:
def get_max_transaction_id() -> int:
    connection: sqlite3.Connection = sqlite3.connect(MMEX_PATH)
    cursor: sqlite3.Cursor = connection.cursor()
    results: list = cursor.execute(
        """
        SELECT
            MAX(TRANSID)
        FROM
            CHECKINGACCOUNT_V1
        """
    ).fetchone()

    return results[0]

def get_column_names_table(table_name: str) -> list[str]:
    connection: sqlite3.Connection = sqlite3.connect(MMEX_PATH)
    cursor: sqlite3.Cursor = connection.cursor()
    cursor = cursor.execute(
        f"""
        SELECT
            *
        FROM
            {table_name}
        LIMIT 1
        """
    )
    column_names: list[str] = [description[0] for description in cursor.description]
    connection.close()
    return column_names

def load_accounts() -> pd.DataFrame:
    connection: sqlite3.Connection = sqlite3.connect(MMEX_PATH)
    cursor: sqlite3.Cursor = connection.cursor()
    results: list = cursor.execute(
        """
        SELECT
            ACCOUNTID, ACCOUNTNAME
        FROM
            ACCOUNTLIST_V1
        """
    ).fetchall()
    
    column_names: list[str] = [description[0] for description in cursor.description]
    connection.close()
    return pd.DataFrame(results, columns=column_names)

def load_transfers(path: str) -> pd.DataFrame:
    transfers: pd.DataFrame = pd.read_csv(path, 
                       names=['Data', 'Stato', 'Tipo', 'Conto', 'Beneficiario', 'Importo', 'Valuta', 'Categoria', 'Sotto-Categoria', 'Note'])
    transfers["Beneficiario"] = transfers["Beneficiario"].str.replace("> ", "")
    return transfers


In [4]:
max_transaction_id: int = get_max_transaction_id()
accounts: pd.DataFrame = load_accounts()

In [8]:
from datetime import datetime

DATETIME_FORMAT = "%Y-%m-%dT%H:%M:%S"



def save_account_trasnfers_to_db(transfers_df: pd.DataFrame):
    payee_id, followup_id, color = -1, -1, -1
    transaction_number, deleted_time = '', ''
    transcode: str = "Transfer"
    status: str = 'R'
    categ_id: int = 102

    transaction_id: int = get_max_transaction_id()
    transfers_list = []
    

    for idx, row in transfers_df.iterrows():
        conto, beneficiario = accounts.loc[accounts.ACCOUNTNAME == row.Conto].ACCOUNTID.to_numpy()[0], accounts.loc[accounts.ACCOUNTNAME == row.Beneficiario].ACCOUNTID.to_numpy()[0]
        note = row.Note
        trans_date: str = row.Data
        amount: float = row.Importo
        update_time: str = datetime.now().strftime(DATETIME_FORMAT)
        transaction_id += 1
        transfers_list.append([transaction_id, conto, beneficiario, payee_id, transcode, amount, status, transaction_number, note, categ_id, trans_date, update_time, deleted_time, followup_id, amount, color])

    
    mmex_transfers_df: pd.DataFrame = pd.DataFrame(transfers_list, columns=get_column_names_table("CHECKINGACCOUNT_V1"))

    with sqlite3.connect(MMEX_PATH) as conn:
        mmex_transfers_df.to_sql("CHECKINGACCOUNT_V1", conn, if_exists="append", index=False)

        

In [9]:
import pathlib

for account_path in pathlib.Path("/media/paolo/Kingston SSD/ledger-to-mmex/data/export").rglob("*transfer.csv"):
    try:
        transfers: pd.DataFrame = load_transfers(account_path)
        save_account_trasnfers_to_db(transfers)
    except Exception as e:
        print(account_path)
        break